In [2]:
## PREAMBLE ##
# Author: Luca Carnegie
# Date: 2025-04-21

# This code cleans the raw FMA metadata and puts it in a usable format for classifier training
# as implemented in 3

# Prerequisites: None

In [2]:
import pandas as pd
import os

# Change working directory to the repo
os.chdir("C:\\Users\\lucac\\Documents\\GitHub\\song-cluster")

In [3]:
# 1. Load raw features with a 3-level header, skip the lone track_id row
features = pd.read_csv(
    'data/raw_data/fma_metadata/features.csv',
    header=[0,1,2],
    skiprows=[3]
)

# 2. Build a fresh DataFrame
features_clean = pd.DataFrame()
features_clean['track_id'] = features.iloc[:, 0]

# 2b. MFCC means 1–16
for i in range(1, 17):
    mfcc_col = next(
        col for col in features.columns
        if col[0].startswith('mfcc') and col[1]=='mean' and int(col[2])==i
    )
    features_clean[f'mfcc_{i}'] = features[mfcc_col]

# 2c. Centroid mean & variance
cent_mean_col = next(c for c in features.columns if c[0].startswith('spectral_centroid') and c[1]=='mean')
cent_std_col  = next(c for c in features.columns if c[0].startswith('spectral_centroid') and c[1]=='std')
features_clean['centroid_mean']     = features[cent_mean_col]
features_clean['centroid_variance'] = features[cent_std_col] ** 2

# 2d. Rolloff mean & variance
roll_mean_col = next(c for c in features.columns if c[0].startswith('spectral_rolloff') and c[1]=='mean')
roll_std_col  = next(c for c in features.columns if c[0].startswith('spectral_rolloff') and c[1]=='std')
features_clean['rolloff_mean']     = features[roll_mean_col]
features_clean['rolloff_variance'] = features[roll_std_col] ** 2

# 2e. ZCR mean & variance
zcr_mean_col = next(c for c in features.columns if c[0].startswith('zcr') and c[1]=='mean')
zcr_std_col  = next(c for c in features.columns if c[0].startswith('zcr') and c[1]=='std')
features_clean['zcr_mean']      = features[zcr_mean_col]
features_clean['zcr_variance']  = features[zcr_std_col] ** 2


# 3. Load tracks metadata (2-level header, skip the track_id row)
tracks = pd.read_csv(
    'data/raw_data/fma_metadata/tracks.csv',
    header=[0,1],
    skiprows=[2]
)

# 4. Dynamically find the right MultiIndex keys
track_id_col = tracks.columns[0]
title_col    = next(col for col in tracks.columns if col[1] == 'title')
artist_col   = next(col for col in tracks.columns if col[1] == 'name')
genre_col    = next(col for col in tracks.columns if col[1] == 'genre_top')

# 5. Build tracks_clean
tracks_clean = pd.DataFrame({
    'track_id':    tracks[track_id_col],
    'title':       tracks[title_col],
    'artist_name': tracks[artist_col],
    'genre_top':   tracks[genre_col],
})

# 6. Merge & drop rows missing genre label
merged = (
    pd.merge(tracks_clean, features_clean, on='track_id', how='inner')
      .dropna(subset=['genre_top'])
)

# 6.5 Drop any rows with missing values in the features
merged = merged.dropna()

merged.to_csv('data/analysis_data/classifier_data.csv', index=False)

# 7. Limit genres to 10 and discard the rest
merged['genre_top'] = merged['genre_top'].where(
    merged['genre_top'].isin(merged['genre_top'].value_counts().nlargest(10).index),
    'other'
)

# 8. Drop rows with 'other' genre
merged = merged[merged['genre_top'] != 'other']

# get counts of each genre
genre_counts = merged['genre_top'].value_counts()
print(genre_counts)


# save the cleaned data
merged.to_csv('data/analysis_data/classifier_data_top10.csv', index=False)

genre_top
Rock             14155
Experimental     10544
Electronic        9260
Hip-Hop           3536
Folk              2773
Pop               2325
Instrumental      2070
International     1378
Classical         1212
Jazz               564
Name: count, dtype: int64


In [28]:
# how many rows don't have a value in genre_top in in tracks.csv? 
no_genre_songs = tracks[tracks['track.7'].isna()].shape[0]
print("Number of rows without a value in tracks: ", no_genre_songs)

# how many rows are there in the tracks dataframe?
number_of_rows = tracks.shape[0]
print("Number of rows in tracks dataframe: ", number_of_rows)

# how many analysable rows are there, total (i.e. have a genre [all features] were extracted)
print("Number of analysable rows in tracks dataframe: ", number_of_rows - no_genre_songs)

Number of rows without a value in tracks:  56977
Number of rows in tracks dataframe:  106576
Number of analysable rows in tracks dataframe:  49599


In [48]:
# print the full dataframe, with all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

test_tracks = pd.read_csv("data/analysis_data/classifier_data.csv", header=[0,1], skiprows=[2])

# Print the first 5 rows of the tracks dataframe
print(test_tracks.shape)

(49596, 26)


In [ ]:
# Get me the first ten rows of features.csv and genres.csv and export them as a csv

os.chdir("C:\\Users\\lucac\\Documents\\GitHub\\song-cluster")

features.head(10).to_csv("data/raw_data/features_head.csv", index=False)
tracks.head(10).to_csv("data/raw_data/tracks_head.csv", index=False)